In [9]:
import os

import matplotlib.pyplot as plt
import numpy as np

"Machine learning tools"
import pickle

from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.neighbors import KNeighborsClassifier

from classification.datasets import Dataset
from classification.utils.audio_student import AudioUtil, Feature_vector_DS

from classification.utils.plots import (
    plot_decision_boundaries,
    plot_specgram,
    show_confusion_matrix,
)
from classification.utils.utils import accuracy

In [10]:
np.random.seed(0)

In [11]:
### TO RUN
dataset = Dataset()
classnames = dataset.list_classes()

print("\n".join(classnames))

chainsaw
fire
fireworks
gunshot


In [12]:
### TO RUN
fm_dir = "data/feature_matrices/"  # where to save the features matrices
new_dataset_dir = "src/classification/datasets/new_dataset/melvecs/"
model_dir = "data/models/random_forest"  # where to save the models
os.makedirs(fm_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)

In [13]:
### TO RUN

"Creation of the dataset"
myds = Feature_vector_DS(dataset, Nft=512, nmel=20, duration=950, shift_pct=0.0)

"Some attributes..."
myds.nmel
myds.duration
myds.shift_pct
myds.sr
myds.data_aug
myds.ncol


idx = 0


In [14]:

import numpy as np
from sklearn.calibration import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler

data_aug_factor = 1
featveclen = len(myds["fire", 0, "", ""])  # Same for all classes
classnames = ["chainsaw", "fire", "fireworks", "gunshot"]  # Or wherever you store class names
nclass = len(classnames)

# Determine number of samples per class
naudio_per_class = {cls: len(dataset.files.get(cls, [])) for cls in classnames}
print(naudio_per_class)

# Allocate feature matrix
total_samples_basic = sum(naudio_per_class[c] for c in classnames)

X_train = []
X_test = []
y_train = []
y_test = []

for class_idx, classname in enumerate(classnames):
    for i in range(naudio_per_class[classname]):
        featvec = myds[classname, i, "", ""]
        if i < naudio_per_class[classname] * 0.8:
            X_train.append(featvec)
            y_train.append(classname)
        else:
            X_test.append(featvec)
            y_test.append(classname)

# Split the dataset into training and testing sets
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

# Normalization of the data
scaler = StandardScaler()
X_train_norm = scaler.fit_transform(X_train)
X_test_norm = scaler.transform(X_test)
with open(os.path.join(model_dir, f"scaler.pkl"), "wb") as f:
    pickle.dump(scaler, f)

label_encoder           = LabelEncoder()
y_train     = label_encoder.fit_transform(y_train)
y_test      = label_encoder.transform(y_test)
with open(os.path.join(model_dir, f"label_encoder.pkl"), "wb") as f:
    pickle.dump(label_encoder, f)

# Save the feature matrix and labels
np.save(os.path.join(fm_dir, "X_train.npy"), X_train)
np.save(os.path.join(fm_dir, "X_test.npy"), X_test)
np.save(os.path.join(fm_dir, "y_train.npy"), y_train)
np.save(os.path.join(fm_dir, "y_test.npy"), y_test)
np.save(os.path.join(fm_dir, "X_train_norm.npy"), X_train_norm)
np.save(os.path.join(fm_dir, "X_test_norm.npy"), X_test_norm)

print(f"Shape of the training matrix : {X_train.shape}")
print(f"Shape of the test matrix : {X_test.shape}")

{'chainsaw': 315, 'fire': 303, 'fireworks': 315, 'gunshot': 40}


Shape of the training matrix : (779, 400)
Shape of the test matrix : (194, 400)


We can now create a new augmented dataset and observe if the classification results improve. 

In [ ]:
### AUGMENTED DATASET
TEST_WITH_AUGMENTED_FV = False

list_augmentation = ["original", "noise", "shifting"]
myds.mod_data_aug(list_augmentation)
print("Number of transformations : ", myds.data_aug_factor)

n_bands = 20
frames = 20
# Préparer les splits
X_train_list, y_train_list = [], []
X_test_list,  y_test_list  = [], []

for classname in classnames:
    n = naudio_per_class[classname]

    # Création des indices de base pour les sons originaux
    limit = round(n*0.8)
    train_idx = list(range(limit))
    test_idx  = list(range(limit+1, n))
    
    for i in train_idx:
        for aug in list_augmentation:
            featvec = myds[classname, i, aug, ""]
            X_train_list.append(featvec)
            y_train_list.append(classname)

    for i in test_idx:
        if TEST_WITH_AUGMENTED_FV:
            for aug in list_augmentation:
                featvec = myds[classname, i, aug, ""]
                X_test_list.append(featvec)
                y_test_list.append(classname)
        else:
            featvec = myds[classname, i, "", ""]
            X_test_list.append(featvec)
            y_test_list.append(classname)

# Conversion en tableaux numpy
X_train_aug = np.array(X_train_list)
y_train_aug = np.array(y_train_list, dtype=object)

X_test_aug = np.array(X_test_list)
y_test_aug = np.array(y_test_list, dtype=object)

# --- 1) Z-SCORE GLOBAL -------------------------------
scaler_global = StandardScaler().fit(X_train_aug)
X_train_aug_norm_zscore_global = scaler_global.transform(X_train_aug)
X_test_aug_norm_zscore_global  = scaler_global.transform(X_test_aug)

with open(os.path.join(model_dir, "scaler_aug_zscore_global.pkl"), "wb") as f:
    pickle.dump(scaler_global, f)
np.save(os.path.join(fm_dir, "X_train_aug_norm_zscore_global.npy"), X_train_aug_norm_zscore_global)
np.save(os.path.join(fm_dir, "X_test_aug_norm_zscore_global.npy"),  X_test_aug_norm_zscore_global)

# --- 2) Z-SCORE PAR BANDE MEL ------------------------------------
X_train_resh = X_train_aug.reshape(-1, n_bands, frames)
X_test_resh  = X_test_aug.reshape(-1, n_bands, frames)

mean_band = X_train_resh.mean(axis=(0, 2))          # (20,)
std_band  = X_train_resh.std(axis=(0, 2)) + 1e-8    # (20,)

X_train_aug_norm_zscore_band = ((X_train_resh - mean_band[None,:,None])
                           / std_band[None,:,None]).reshape(-1, n_bands*frames)
X_test_aug_norm_zscore_band  = ((X_test_resh  - mean_band[None,:,None])
                           / std_band[None,:,None]).reshape(-1, n_bands*frames)

np.save(os.path.join(model_dir, "zscore_band_mean.npy"), mean_band)
np.save(os.path.join(model_dir, "zscore_band_std.npy"),  std_band)
np.save(os.path.join(fm_dir, "X_train_aug_norm_zscore_band.npy"), X_train_aug_norm_zscore_band)
np.save(os.path.join(fm_dir, "X_test_aug_norm_zscore_band.npy"),  X_test_aug_norm_zscore_band)

# --- 3) MIN-MAX PAR BANDE MEL ------------------------------------
min_band = X_train_resh.min(axis=(0, 2))
max_band = X_train_resh.max(axis=(0, 2)) + 1e-8

X_train_aug_norm_minmax = ((X_train_resh - min_band[None,:,None])
                           / (max_band - min_band)[None,:,None]).reshape(-1, n_bands*frames)
X_test_aug_norm_minmax  = ((X_test_resh  - min_band[None,:,None])
                           / (max_band - min_band)[None,:,None]).reshape(-1, n_bands*frames)

np.save(os.path.join(model_dir, "min_band.npy"), min_band)
np.save(os.path.join(model_dir, "max_band.npy"), max_band)
np.save(os.path.join(fm_dir, "X_train_aug_norm_minmax.npy"), X_train_aug_norm_minmax)
np.save(os.path.join(fm_dir, "X_test_aug_norm_minmax.npy"),  X_test_aug_norm_minmax)

# --- 4) GLOBAL MAX NORMALIZATION -------------------------------
# Chaque spectrogramme est divisé par son max propre (par ligne)
X_train_aug_norm_max = X_train_aug.copy().astype(np.float32)
X_test_aug_norm_max  = X_test_aug.copy().astype(np.float32)

X_train_max = X_train_aug_norm_max.max(axis=1, keepdims=True) + 1e-8 # éviter la division par 0
X_test_max  = X_test_aug_norm_max.max(axis=1, keepdims=True) + 1e-8

X_train_aug_norm_max /= X_train_max
X_test_aug_norm_max  /= X_test_max

np.save(os.path.join(fm_dir, "X_train_aug_norm_max.npy"), X_train_aug_norm_max)
np.save(os.path.join(fm_dir, "X_test_aug_norm_max.npy"),  X_test_aug_norm_max)

# --- 5) L2 NORMALIZATION -------------------------------
X_train_aug_norm_l2 = X_train_aug.copy().astype(np.float32)
X_test_aug_norm_l2  = X_test_aug.copy().astype(np.float32)

X_train_l2 = np.sqrt(np.sum(X_train_aug_norm_l2**2, axis=1, keepdims=True)) + 1e-8
X_test_l2  = np.sqrt(np.sum(X_test_aug_norm_l2**2, axis=1, keepdims=True)) + 1e-8

X_train_aug_norm_l2 /= X_train_l2
X_test_aug_norm_l2  /= X_test_l2

np.save(os.path.join(fm_dir, "X_train_aug_norm_l2.npy"), X_train_aug_norm_l2)
np.save(os.path.join(fm_dir, "X_test_aug_norm_l2.npy"),  X_test_aug_norm_l2)


# Label encoding
label_encoder_aug = LabelEncoder()
y_train_aug     = label_encoder_aug.fit_transform(y_train_aug)
y_test_aug      = label_encoder_aug.transform(y_test_aug)
with open(os.path.join(model_dir, f"label_encoder_aug.pkl"), "wb") as f:
    pickle.dump(label_encoder_aug, f)

np.save(os.path.join(fm_dir, "X_train_aug.npy"), X_train_aug)
np.save(os.path.join(fm_dir, "X_test_aug.npy"), X_test_aug)
np.save(os.path.join(fm_dir, "y_train_aug.npy"), y_train_aug)
np.save(os.path.join(fm_dir, "y_test_aug.npy"), y_test_aug)

print(f"Shape of the training matrix : {X_train_aug.shape}")
print(f"Shape of the test matrix : {X_test_aug.shape}")
print(f"------------------------------------------------------------")
print(f"Transformations: {list_augmentation}.")


Number of transformations :  2
Shape of the training matrix : (1556, 400)
Shape of the test matrix : (191, 400)
------------------------------------------------------------
Transformations: ['original', 'shifting'].


FINAL MODEL SAVE

In [16]:
import os
import numpy as np
import pickle

import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, precision_score, recall_score

# =========================
# 1) HYPERPARAMS & PATHS
# =========================
# Define or ensure these variables exist:
# fm_dir = "/path/to/features/"
# model_dir = "/path/to/save/models/"

n_estimators = 400
max_depth = 20
min_samples_split = 5
min_samples_leaf = 2
random_state = 42

# =========================
# 2) LOAD DATA
# =========================
X_train_aug = np.load(os.path.join(fm_dir, "X_train_aug.npy"))
X_test_aug = np.load(os.path.join(fm_dir, "X_test_aug.npy"))
y_train_aug = np.load(os.path.join(fm_dir, "y_train_aug.npy"))
y_test_aug = np.load(os.path.join(fm_dir, "y_test_aug.npy"))

X_train_aug_norm = np.load(os.path.join(fm_dir, "X_train_aug_norm_max.npy"))
X_test_aug_norm = np.load(os.path.join(fm_dir, "X_test_aug_norm_max.npy"))


X_train = np.load(os.path.join(fm_dir, "X_train.npy"))
X_test = np.load(os.path.join(fm_dir, "X_test.npy"))
y_train = np.load(os.path.join(fm_dir, "y_train.npy"))
y_test = np.load(os.path.join(fm_dir, "y_test.npy"))

X_train_norm = np.load(os.path.join(fm_dir, "X_train_norm.npy"))
X_test_norm = np.load(os.path.join(fm_dir, "X_test_norm.npy"))



# FROM HERE MODIFY PLEASE CHAT GPT


def save_obj(obj, name):
    with open(os.path.join(model_dir, f"{name}.pkl"), "wb") as f:
        pickle.dump(obj, f)

def build_rf():
    return RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=random_state,
    )

def evaluate(model, Xte, yte, tag):
    pred   = model.predict(Xte)
    acc    = (pred == yte).mean()
    cv_acc = cross_val_score(model, Xte, yte, cv=5, scoring="accuracy").mean()

    classes = np.unique(yte)
    prec = precision_score(yte, pred, average=None, labels=classes)
    rec  = recall_score(yte, pred, average=None, labels=classes)
    cm   = confusion_matrix(yte, pred, labels=classes)
    per_cls_acc = [cm[i, i] / cm[i, :].sum() for i in range(len(classes))]

    print(f"\n=== {tag} ===")
    print(f"Overall accuracy : {acc:.4f}  |  CV accuracy : {cv_acc:.4f}")
    for i, c in enumerate(classes):
        print(f"Class {c}: P={prec[i]:.3f} R={rec[i]:.3f} Acc={per_cls_acc[i]:.3f}")

# ------------------------------------------------------------------
# 3) SCÉNARIOS A → H
# ------------------------------------------------------------------
models = {}

# A : PCA, no‑aug, no‑norm
pca_A   = PCA(n_components=0.99).fit(X_train)
Xtr_A   = pca_A.transform(X_train)
Xte_A   = pca_A.transform(X_test)
rf_A    = build_rf().fit(Xtr_A, y_train)
save_obj(pca_A, "pca_A"); save_obj(rf_A, "rf_A")
models["A"] = (rf_A, Xte_A, y_test)

# B : noPCA, no‑aug, no‑norm
rf_B = build_rf().fit(X_train, y_train)
save_obj(rf_B, "rf_B")
models["B"] = (rf_B, X_test, y_test)

# C : PCA, aug, no‑norm
pca_C  = PCA(n_components=0.99).fit(X_train_aug)
Xtr_C  = pca_C.transform(X_train_aug)
Xte_C  = pca_C.transform(X_test_aug)
rf_C   = build_rf().fit(Xtr_C, y_train_aug)
save_obj(pca_C, "pca_C"); save_obj(rf_C, "rf_C")
models["C"] = (rf_C, Xte_C, y_test_aug)

# D : noPCA, aug, no‑norm
rf_D = build_rf().fit(X_train_aug, y_train_aug)
save_obj(rf_D, "rf_D")
models["D"] = (rf_D, X_test_aug, y_test_aug)

# E : alias de B (noPCA, no‑aug, no‑norm)
models["E"] = models["B"]

# F : PCA, no‑aug, norm
pca_F  = PCA(n_components=0.99).fit(X_train_norm)
Xtr_F  = pca_F.transform(X_train_norm)
Xte_F  = pca_F.transform(X_test_norm)
rf_F   = build_rf().fit(Xtr_F, y_train)
save_obj(pca_F, "pca_F"); save_obj(rf_F, "rf_F")
models["F"] = (rf_F, Xte_F, y_test)

# G : PCA, aug, norm
pca_G  = PCA(n_components=0.99).fit(X_train_aug_norm)
Xtr_G  = pca_G.transform(X_train_aug_norm)
Xte_G  = pca_G.transform(X_test_aug_norm)
rf_G   = build_rf().fit(Xtr_G, y_train_aug)
save_obj(pca_G, "pca_G"); save_obj(rf_G, "rf_G")
models["G"] = (rf_G, Xte_G, y_test_aug)

# H : noPCA, aug, norm
rf_H = build_rf().fit(X_train_aug_norm, y_train_aug)
save_obj(rf_H, "rf_H")
models["H"] = (rf_H, X_test_aug_norm, y_test_aug)

# ------------------------------------------------------------------
# 4) ÉVALUATION GLOBALE
# ------------------------------------------------------------------
scenario_names = {
    "A": "PCA NOAUG NONORM",
    "B": "NOPCA NOAUG NONORM",
    "C": "PCA AUG NONORM",
    "D": "NOPCA AUG NONORM",
    "E": "NOPCA NOAUG NONORM (alias)",
    "F": "PCA NOAUG NORM",
    "G": "PCA AUG NORM",
    "H": "NOPCA AUG NORM",
}

for tag, (model, Xeval, yeval) in models.items():
    evaluate(model, Xeval, yeval, f"Scenario {tag}: {scenario_names[tag]}")



=== Scenario A: PCA NOAUG NONORM ===
Overall accuracy : 0.6031  |  CV accuracy : 0.5829
Class 0: P=0.658 R=0.794 Acc=0.794
Class 1: P=0.691 R=0.633 Acc=0.633
Class 2: P=0.424 R=0.397 Acc=0.397
Class 3: P=1.000 R=0.500 Acc=0.500

=== Scenario B: NOPCA NOAUG NONORM ===
Overall accuracy : 0.7629  |  CV accuracy : 0.7833
Class 0: P=0.833 R=0.794 Acc=0.794
Class 1: P=0.769 R=0.833 Acc=0.833
Class 2: P=0.678 R=0.635 Acc=0.635
Class 3: P=0.800 R=1.000 Acc=1.000

=== Scenario C: PCA AUG NONORM ===
Overall accuracy : 0.6230  |  CV accuracy : 0.6020
Class 0: P=0.686 R=0.774 Acc=0.774
Class 1: P=0.702 R=0.667 Acc=0.667
Class 2: P=0.421 R=0.387 Acc=0.387
Class 3: P=1.000 R=1.000 Acc=1.000

=== Scenario D: NOPCA AUG NONORM ===
Overall accuracy : 0.8010  |  CV accuracy : 0.7853
Class 0: P=0.862 R=0.806 Acc=0.806
Class 1: P=0.831 R=0.900 Acc=0.900
Class 2: P=0.724 R=0.677 Acc=0.677
Class 3: P=0.700 R=1.000 Acc=1.000

=== Scenario E: NOPCA NOAUG NONORM (alias) ===
Overall accuracy : 0.7629  |  CV acc

/home/ppasture/master_projet_elec/LELEC210X/.venv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



=== Scenario G: PCA AUG NORM ===
Overall accuracy : 0.6440  |  CV accuracy : 0.5293
Class 0: P=0.615 R=0.774 Acc=0.774
Class 1: P=0.648 R=0.767 Acc=0.767
Class 2: P=0.690 R=0.468 Acc=0.468
Class 3: P=0.000 R=0.000 Acc=0.000

=== Scenario H: NOPCA AUG NORM ===
Overall accuracy : 0.7330  |  CV accuracy : 0.6594
Class 0: P=0.770 R=0.758 Acc=0.758
Class 1: P=0.685 R=0.833 Acc=0.833
Class 2: P=0.740 R=0.597 Acc=0.597
Class 3: P=0.857 R=0.857 Acc=0.857


MEAN ACCURACY ON 20 ITERATIONS

In [17]:
"""
import os
import numpy as np
import pickle
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from classification.utils.utils import accuracy

NORMALIZATION = True
TRANSFORMATION = True

# Ensure dataset (X_aug, y_aug) exists

if TRANSFORMATION:
    try:
        X = X_basic_aug
        y = y_basic_aug
    except NameError:
        raise ValueError("X_aug and y_aug must be defined before running this script.")
else:
    try:
        X = X_basic
        y = y_basic
    except NameError:
        raise ValueError("X and y must be defined before running this script.")
if NORMALIZATION:
    X = np.array([x/np.linalg.norm(x) if np.linalg.norm(x) != 0 else x for x in X])
    


# Number of iterations
num_iterations = 20

# Lists to store scores
accuracy_scores = []
cv_accuracy_scores = []

for i in range(num_iterations):
    print(f"\nIteration {i + 1}/{num_iterations}")

    # Split the dataset into training and testing subsets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, stratify=y, random_state=i  # Different splits per iteration
    )

    # Train the Random Forest model
    model = RandomForestClassifier(
        n_estimators=400,
        max_depth=20,
        min_samples_split=5,
        min_samples_leaf=2,
        random_state=i  # Different initialization per iteration
    )
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Compute overall accuracy
    test_accuracy = accuracy(y_pred, y_test)
    accuracy_scores.append(test_accuracy)

    # Perform cross-validation on the training set
    cv_scores = cross_val_score(model, X_train, y_train, cv=10, scoring='accuracy')
    mean_cv_accuracy = np.mean(cv_scores)
    cv_accuracy_scores.append(mean_cv_accuracy)

    print(f"Test Accuracy: {test_accuracy:.4f} | Mean CV Accuracy: {mean_cv_accuracy:.4f}")

# Compute overall statistics
mean_test_accuracy = np.mean(accuracy_scores)
std_test_accuracy = np.std(accuracy_scores)

mean_cv_accuracy = np.mean(cv_accuracy_scores)
std_cv_accuracy = np.std(cv_accuracy_scores)

# Print final results
print("\n=== FINAL RESULTS AFTER 20 ITERATIONS ===")
print(f"Mean Test Accuracy: {mean_test_accuracy:.4f} ± {std_test_accuracy:.4f}")
print(f"Mean Cross-Validation Accuracy: {mean_cv_accuracy:.4f} ± {std_cv_accuracy:.4f}")

"""

'\nimport os\nimport numpy as np\nimport pickle\nimport matplotlib.pyplot as plt\nfrom sklearn.ensemble import RandomForestClassifier\nfrom sklearn.model_selection import train_test_split, cross_val_score\nfrom sklearn.metrics import confusion_matrix, precision_score, recall_score\nfrom classification.utils.utils import accuracy\n\nNORMALIZATION = True\nTRANSFORMATION = True\n\n# Ensure dataset (X_aug, y_aug) exists\n\nif TRANSFORMATION:\n    try:\n        X = X_basic_aug\n        y = y_basic_aug\n    except NameError:\n        raise ValueError("X_aug and y_aug must be defined before running this script.")\nelse:\n    try:\n        X = X_basic\n        y = y_basic\n    except NameError:\n        raise ValueError("X and y must be defined before running this script.")\nif NORMALIZATION:\n    X = np.array([x/np.linalg.norm(x) if np.linalg.norm(x) != 0 else x for x in X])\n    \n\n\n# Number of iterations\nnum_iterations = 20\n\n# Lists to store scores\naccuracy_scores = []\ncv_accuracy_sc